miStudio Complete End-to-End Pipeline (2025)

This notebook demonstrates the complete miStudio interpretability workflow:
1. Health checks for all services
2. Train a Sparse Autoencoder (SAE) on Microsoft Phi-4 at layer 30
3. Find and analyze features using miStudioFind
4. Generate explanations using miStudioExplain  
5. Score feature importance using miStudioScore
6. Export and analyze complete results

Prerequisites:
- All miStudio services running on their designated ports
- GPU with sufficient memory (>= 16GB recommended for Phi-4)
- Training data prepared (webtext corpus)

In [22]:
#!/usr/bin/env python3
# miStudio Complete Workflow - Phi-4 SAE Training and Analysis
# This script orchestrates the entire workflow for training, finding, explaining, and scoring a model using miStudio services.

import requests
import json
import time
import os
import sys
import pandas as pd
from pathlib import Path
from typing import Dict, Any, Optional
import zipfile
import urllib.request
from datetime import datetime

# Configuration
SERVICE_PORTS = {
    'train': 8001,
    'find': 8002, 
    'explain': 8003,
    'score': 8004
}

BASE_URL = "http://localhost"
SERVICE_URLS = {
    service: f"{BASE_URL}:{port}" 
    for service, port in SERVICE_PORTS.items()
}

# Model and training configuration
MODEL_NAME = "microsoft/Phi-4"
TARGET_LAYER = 30  # Layer 30 for Phi-4
WEBTEXT_CORPUS_URL = "https://huggingface.co/datasets/stas/openwebtext-10k/resolve/main/plain_text/train-00000-of-00001.parquet"
CORPUS_FILENAME = "webtext_corpus.txt"

# Output directories
OUTPUT_DIR = Path("./mistudio_phi4_results")
OUTPUT_DIR.mkdir(exist_ok=True)

# Global variables to store results between steps
training_job_id = None
find_job_id = None
explain_job_id = None
score_job_id = None

print("🚀 miStudio Complete Workflow - Phi-4 SAE Training and Analysis")
print("=" * 70)
print(f"Model: {MODEL_NAME}")
print(f"Target Layer: {TARGET_LAYER}")
print(f"Output Directory: {OUTPUT_DIR.absolute()}")
print(f"Service URLs configured:")
for service, url in SERVICE_URLS.items():
    print(f"  - {service.capitalize()}: {url}")
print("=" * 70)

# =============================================================================
# Utility Functions
# =============================================================================

def check_service_health(service_name: str, url: str) -> bool:
    """Check if a service is healthy and responsive."""
    try:
        response = requests.get(f"{url}/health", timeout=10)
        if response.status_code == 200:
            health_data = response.json()
            print(f"✅ {service_name.capitalize()} service: {health_data.get('status', 'healthy')}")
            return True
        else:
            print(f"❌ {service_name.capitalize()} service: HTTP {response.status_code}")
            return False
    except requests.RequestException as e:
        print(f"❌ {service_name.capitalize()} service: Connection failed - {e}")
        return False

def wait_for_job_completion(service_url: str, job_id: str, service_name: str, 
                          max_wait_minutes: int = 120) -> Dict[str, Any]:
    """Wait for a job to complete and return the final status."""
    start_time = time.time()
    max_wait_seconds = max_wait_minutes * 60
    
    print(f"⏳ Waiting for {service_name} job {job_id} to complete...")
    
    while time.time() - start_time < max_wait_seconds:
        try:
            response = requests.get(f"{service_url}/api/v1/{service_name.lower()}/{job_id}/status")
            if response.status_code == 200:
                status_data = response.json()
                status = status_data.get('status')
                
                if status == 'completed':
                    print(f"✅ {service_name} job completed successfully!")
                    return status_data
                elif status == 'failed':
                    print(f"❌ {service_name} job failed!")
                    print(f"Error: {status_data.get('error', 'Unknown error')}")
                    return status_data
                elif status in ['running', 'queued']:
                    progress = status_data.get('progress', {})
                    if 'percentage' in progress:
                        print(f"🔄 {service_name} progress: {progress['percentage']:.1f}% - {progress.get('message', '')}")
                    else:
                        print(f"🔄 {service_name} status: {status}")
                    time.sleep(30)  # Check every 30 seconds
                else:
                    print(f"🔄 {service_name} status: {status}")
                    time.sleep(30)
            else:
                print(f"⚠️ Failed to get {service_name} status: HTTP {response.status_code}")
                time.sleep(30)
        except requests.RequestException as e:
            print(f"⚠️ Error checking {service_name} status: {e}")
            time.sleep(30)
    
    print(f"⏰ Timeout waiting for {service_name} job to complete after {max_wait_minutes} minutes")
    return {'status': 'timeout'}

def download_webtext_corpus() -> str:
    """Download and prepare the webtext corpus for training."""
    corpus_path = OUTPUT_DIR / CORPUS_FILENAME
    
    if corpus_path.exists():
        print(f"📁 Webtext corpus already exists: {corpus_path}")
        return str(corpus_path)
    
    print("📥 Downloading webtext corpus...")
    
    try:
        # Download parquet file
        parquet_path = OUTPUT_DIR / "webtext.parquet"
        urllib.request.urlretrieve(WEBTEXT_CORPUS_URL, parquet_path)
        
        # Convert to text format
        import pandas as pd
        df = pd.read_parquet(parquet_path)
        
        with open(corpus_path, 'w', encoding='utf-8') as f:
            for text in df['text'].head(1000):  # Use first 1000 texts
                if text and len(text.strip()) > 50:  # Filter short texts
                    f.write(text.strip() + '\n')
        
        # Clean up parquet file
        parquet_path.unlink()
        
        print(f"✅ Webtext corpus created: {corpus_path}")
        return str(corpus_path)
        
    except Exception as e:
        print(f"⚠️ Error downloading corpus: {e}")
        print("🔄 Creating fallback corpus...")
        
        # Create fallback corpus
        fallback_texts = [
            "Transformers have revolutionized natural language understanding.",
            "Sparse autoencoders help us understand what neural networks learn.",
            "Feature interpretability is crucial for AI safety and alignment.",
        ] * 100  # Repeat to create more content
        
        with open(corpus_path, 'w', encoding='utf-8') as f:
            for text in fallback_texts:
                f.write(text + '\n')
        
        print(f"✅ Fallback corpus created: {corpus_path}")
        return str(corpus_path)

🚀 miStudio Complete Workflow - Phi-4 SAE Training and Analysis
Model: microsoft/Phi-4
Target Layer: 30
Output Directory: /home/sean/app/miStudio/mistudio_phi4_results
Service URLs configured:
  - Train: http://localhost:8001
  - Find: http://localhost:8002
  - Explain: http://localhost:8003
  - Score: http://localhost:8004


In [23]:
# =============================================================================
# Step 0: Service Health Checks
# =============================================================================

print("\n🏥 STEP 0: SERVICE HEALTH CHECKS")
print("-" * 50)

all_healthy = True
for service_name, service_url in SERVICE_URLS.items():
    is_healthy = check_service_health(service_name, service_url)
    if not is_healthy:
        all_healthy = False

if not all_healthy:
    print("\n❌ Some services are not healthy.")
    print("Please ensure all miStudio services are running before proceeding.")
    print("Start services with:")
    for service in SERVICE_PORTS.keys():
        print(f"  cd ~/app/miStudio/services/miStudio{service.capitalize()}")
        print(f"  ./scripts/dev.sh")
    sys.exit(1)

print("\n✅ All services are healthy! Proceeding with workflow...")


🏥 STEP 0: SERVICE HEALTH CHECKS
--------------------------------------------------
✅ Train service: healthy
✅ Find service: healthy
✅ Explain service: healthy
✅ Score service: healthy

✅ All services are healthy! Proceeding with workflow...


In [24]:
# =============================================================================
# Step 1: Prepare Training Data
# =============================================================================

print("\n📋 STEP 1: TRAINING DATA PREPARATION")
print("-" * 50)

corpus_path = download_webtext_corpus()


📋 STEP 1: TRAINING DATA PREPARATION
--------------------------------------------------
📁 Webtext corpus already exists: mistudio_phi4_results/webtext_corpus.txt


In [25]:
# =============================================================================
# Step 2: Train SAE
# =============================================================================

print("\n🏋️ STEP 2: SAE TRAINING")
print("-" * 50)

train_request = {
    "model_name": MODEL_NAME,
    "corpus_file": CORPUS_FILENAME,
    "layer_number": TARGET_LAYER,
    "hidden_dim": 1024,
    "sparsity_coeff": 1e-3,
    "learning_rate": 1e-4,
    "batch_size": 8,  # Conservative batch size for Phi-4
    "max_epochs": 20,
    "min_loss": 0.01,
    "max_sequence_length": 512,
    "gpu_id": 0  # Use first GPU
}

print(f"🚀 Starting SAE training job...")
print(f"📋 Configuration:")
print(f"  - Model: {MODEL_NAME}")
print(f"  - Target Layer: {TARGET_LAYER}")
print(f"  - Hidden Dimensions: {train_request['hidden_dim']}")
print(f"  - Batch Size: {train_request['batch_size']}")
print(f"  - Max Epochs: {train_request['max_epochs']}")

try:
    response = requests.post(f"{SERVICE_URLS['train']}/api/v1/train", json=train_request)
    
    if response.status_code in [200, 202]:  # Accept both 200 and 202 as success
        train_result = response.json()
        training_job_id = train_result['job_id']
        print(f"✅ Training job started: {training_job_id}")
        print(f"📊 Job Details:")
        print(f"  - Status: {train_result.get('status')}")
        print(f"  - Model: {train_result.get('model_name')}")
        print(f"  - Memory Check: {train_result.get('memory_check')}")
        print(f"  - Optimizations: {train_result.get('optimizations_applied')}")
        
        # Wait for training completion
        final_status = wait_for_job_completion(
            SERVICE_URLS['train'], training_job_id, 'Train', max_wait_minutes=180
        )
        
        if final_status.get('status') == 'completed':
            print(f"🎯 Training completed successfully!")
            
            # Get training results
            result_response = requests.get(f"{SERVICE_URLS['train']}/api/v1/train/{training_job_id}/result")
            if result_response.status_code == 200:
                training_results = result_response.json()
                print(f"📊 Training Results:")
                print(f"  - Final Loss: {training_results.get('final_loss', 'N/A')}")
                print(f"  - Epochs Completed: {training_results.get('training_stats', {}).get('total_epochs', 'N/A')}")
                print(f"  - Features: {training_results.get('feature_count', 'N/A')}")
                print(f"  - Ready for Find: {training_results.get('ready_for_find_service', False)}")
            else:
                print(f"⚠️ Could not retrieve training results")
        else:
            print(f"❌ Training failed or timed out")
            sys.exit(1)
            
    else:
        print(f"❌ Failed to start training: HTTP {response.status_code}")
        print(response.text)
        sys.exit(1)
        
except Exception as e:
    print(f"❌ Error during training: {e}")
    sys.exit(1)

✅ Training job started: train_20250801_035441_8498
📊 Job Details:
  - Status: queued
  - Model: microsoft/Phi-4
  - Memory Check: passed
  - Optimizations: Applied optimizations for microsoft/Phi-4: True
⏳ Waiting for Train job train_20250801_035441_8498 to complete...
✅ Train job completed successfully!
🎯 Training completed successfully!
📊 Training Results:
  - Final Loss: N/A
  - Epochs Completed: N/A
  - Features: 1024
  - Ready for Find: True


In [26]:
# =============================================================================
# Step 3: Feature Analysis with miStudioFind
# =============================================================================

print("\n🔍 STEP 3: FEATURE ANALYSIS")
print("-" * 50)

find_request = {
    "source_job_id": training_job_id,
    "top_k": 20,
    "coherence_threshold": 0.5,
    "include_statistics": True
}

print(f"🔎 Starting feature analysis...")
print(f"📋 Configuration:")
print(f"  - Source Job: {training_job_id}")
print(f"  - Top K Activations: {find_request['top_k']}")
print(f"  - Coherence Threshold: {find_request['coherence_threshold']}")

try:
    response = requests.post(f"{SERVICE_URLS['find']}/api/v1/find/analyze", json=find_request)
    
    if response.status_code in [202, 200]:  # Accept both 202 and 200 as success
        # Start feature analysis job
        find_result = response.json()
        find_job_id = find_result['job_id']
        print(f"✅ Feature analysis job started: {find_job_id}")
        
        # Wait for analysis completion
        final_status = wait_for_job_completion(
            SERVICE_URLS['find'], find_job_id, 'Find', max_wait_minutes=30
        )
        
        if final_status.get('status') == 'completed':
            print(f"🎯 Feature analysis completed!")
            
            # Get analysis results
            result_response = requests.get(f"{SERVICE_URLS['find']}/api/v1/find/{find_job_id}/results")
            if result_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
                find_results = result_response.json()
                print(f"📊 Analysis Results:")
                print(f"  - Features Analyzed: {find_results.get('summary', {}).get('total_features_analyzed', find_results.get('feature_count', 'N/A'))}")
                print(f"  - Interpretable Features: {find_results.get('summary', {}).get('interpretable_features', 'N/A')}")
                
                # Handle mean coherence with proper formatting
                mean_coherence = find_results.get('summary', {}).get('mean_coherence_score', 'N/A')
                if isinstance(mean_coherence, (int, float)):
                    print(f"  - Mean Coherence: {mean_coherence:.3f}")
                else:
                    print(f"  - Mean Coherence: {mean_coherence}")
                
                print(f"  - Ready for Explain: {find_results.get('ready_for_explain_service', True)}")
            else:
                print(f"⚠️ Could not retrieve analysis results")
        else:
            print(f"❌ Feature analysis failed or timed out")
            sys.exit(1)
            
    else:
        print(f"❌ Failed to start feature analysis: HTTP {response.status_code}")
        print(response.text)
        sys.exit(1)
        
except Exception as e:
    print(f"❌ Error during feature analysis: {e}")
    sys.exit(1)


🔍 STEP 3: FEATURE ANALYSIS
--------------------------------------------------
🔎 Starting feature analysis...
📋 Configuration:
  - Source Job: train_20250801_035441_8498
  - Top K Activations: 20
  - Coherence Threshold: 0.5
✅ Feature analysis job started: find_20250801_035657_79d4cc16
⏳ Waiting for Find job find_20250801_035657_79d4cc16 to complete...
✅ Find job completed successfully!
🎯 Feature analysis completed!
📊 Analysis Results:
  - Features Analyzed: 1024
  - Interpretable Features: N/A
  - Mean Coherence: N/A
  - Ready for Explain: True


In [27]:
# =============================================================================
# Step 4: Generate Explanations with miStudioExplain
# =============================================================================

print("\n💬 STEP 4: EXPLANATION GENERATION")
print("-" * 50)

explain_request = {
    "find_job_id": find_job_id,
    "model": "llama3.2:3b",  # Use available Ollama model
    "max_explanations": 50,
    "quality_threshold": 0.5
}

print(f"🧠 Starting explanation generation...")
print(f"📋 Configuration:")
print(f"  - Source Find Job: {find_job_id}")
print(f"  - Max Explanations: {explain_request['max_explanations']}")
print(f"  - Model: {explain_request['model']}")

try:
    response = requests.post(f"{SERVICE_URLS['explain']}/api/v1/explain", json=explain_request)

    if response.status_code in [202, 200]:  # Accept both 202 and 200 as success
        explain_result = response.json()
        explain_job_id = explain_result['job_id']
        print(f"✅ Explanation generation job started: {explain_job_id}")
        
        # Wait for explanation completion
        final_status = wait_for_job_completion(
            SERVICE_URLS['explain'], explain_job_id, 'Explain', max_wait_minutes=60
        )
        
        if final_status.get('status') == 'completed':
            print(f"🎯 Explanation generation completed!")
            
            # Get explanation results
            result_response = requests.get(f"{SERVICE_URLS['explain']}/api/v1/explain/{explain_job_id}/results")
            if result_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
                explanation_results = result_response.json()
                print(f"📊 Explanation Results:")
                print(f"  - Explanations Generated: {explanation_results.get('total_explanations', 'N/A')}")
                print(f"  - Average Quality Score: {explanation_results.get('average_quality_score', 'N/A')}")
                print(f"  - Processing Time: {explanation_results.get('processing_time_seconds', 'N/A')}s")
            else:
                print(f"⚠️ Could not retrieve explanation results")
        else:
            print(f"❌ Explanation generation failed or timed out")
            sys.exit(1)
            
    else:
        print(f"❌ Failed to start explanation generation: HTTP {response.status_code}")
        print(response.text)
        sys.exit(1)
        
except Exception as e:
    print(f"❌ Error during explanation generation: {e}")
    sys.exit(1)


💬 STEP 4: EXPLANATION GENERATION
--------------------------------------------------
🧠 Starting explanation generation...
📋 Configuration:
  - Source Find Job: find_20250801_035657_79d4cc16
  - Max Explanations: 50
  - Model: llama3.2:3b
✅ Explanation generation job started: explain_20250801_035657_5f5f0d4c
⏳ Waiting for Explain job explain_20250801_035657_5f5f0d4c to complete...
🔄 Explain status: running
✅ Explain job completed successfully!
🎯 Explanation generation completed!
📊 Explanation Results:
  - Explanations Generated: N/A
  - Average Quality Score: N/A
  - Processing Time: N/As


In [28]:
# =============================================================================
# Step 5: Score Features with miStudioScore
# =============================================================================

print("\n📊 STEP 5: FEATURE SCORING")
print("-" * 50)

# Create scoring configuration
scoring_config = {
    "scoring_jobs": [
        {
            "name": "security_relevance",
            "scorer": "relevance_scorer",
            "parameters": {
                "positive_keywords": ["security", "authentication", "encryption", "privacy", "access"],
                "negative_keywords": ["decoration", "style", "cosmetic", "visual"]
            }
        },
        {
            "name": "technical_complexity",
            "scorer": "ablation_scorer",
            "parameters": {
                "threshold": 0.3,
                "analysis_type": "complexity_assessment"
            }
        }
    ]
}

score_request = {
    "source_type": "explain",
    "source_job_id": explain_job_id,
    "scoring_config": scoring_config,
    "output_formats": ["json", "csv"]
}

print(f"🎯 Starting feature scoring...")
print(f"📋 Configuration:")
print(f"  - Source Explain Job: {explain_job_id}")
print(f"  - Scoring Jobs: {len(scoring_config['scoring_jobs'])}")
print(f"  - Scorers: {[job['scorer'] for job in scoring_config['scoring_jobs']]}")

try:
    response = requests.post(f"{SERVICE_URLS['score']}/api/v1/score/analyze", json=score_request)

    if response.status_code in [200, 202]:  # Accept both 200 and 202 as success   
        score_result = response.json()
        score_job_id = score_result['job_id']
        print(f"✅ Feature scoring job started: {score_job_id}")
        
        # Wait for scoring completion
        final_status = wait_for_job_completion(
            SERVICE_URLS['score'], score_job_id, 'Score', max_wait_minutes=30
        )
        
        if final_status.get('status') == 'completed':
            print(f"🎯 Feature scoring completed!")
            
            # Get scoring results
            result_response = requests.get(f"{SERVICE_URLS['score']}/api/v1/score/{score_job_id}/results")
            if result_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
                scoring_results = result_response.json()
                print(f"📊 Scoring Results:")
                print(f"  - Scores Added: {scoring_results.get('scores_added', [])}")
                print(f"  - Total Features: {scoring_results.get('results_summary', {}).get('total_features', 'N/A')}")
                print(f"  - Processing Time: {scoring_results.get('processing_time', 'N/A')}s")
            else:
                print(f"⚠️ Could not retrieve scoring results")
        else:
            print(f"❌ Feature scoring failed or timed out")
            sys.exit(1)
            
    else:
        print(f"❌ Failed to start feature scoring: HTTP {response.status_code}")
        print(response.text)
        sys.exit(1)
        
except Exception as e:
    print(f"❌ Error during feature scoring: {e}")
    sys.exit(1)


📊 STEP 5: FEATURE SCORING
--------------------------------------------------
🎯 Starting feature scoring...
📋 Configuration:
  - Source Explain Job: explain_20250801_035657_5f5f0d4c
  - Scoring Jobs: 2
  - Scorers: ['relevance_scorer', 'ablation_scorer']
✅ Feature scoring job started: score_20250801_035727_04625c7c
⏳ Waiting for Score job score_20250801_035727_04625c7c to complete...
✅ Score job completed successfully!
🎯 Feature scoring completed!
📊 Scoring Results:
  - Scores Added: ['security_relevance', 'technical_complexity']
  - Total Features: 0
  - Processing Time: 0.000618s


In [29]:


# =============================================================================
# Step 6: Results Summary and Export
# =============================================================================

print("\n📋 STEP 6: RESULTS SUMMARY AND EXPORT")
print("-" * 50)

print(f"🎉 Complete miStudio workflow finished successfully!")
print(f"")
print(f"📂 Job IDs:")
print(f"  - Training: {training_job_id}")
print(f"  - Feature Analysis: {find_job_id}")
print(f"  - Explanations: {explain_job_id}")
print(f"  - Feature Scoring: {score_job_id}")
print(f"")
print(f"📁 Results available in shared data directory:")
print(f"  - Training: /data/results/train/{training_job_id}/")
print(f"  - Find: /data/results/find/{find_job_id}/")
print(f"  - Explain: /data/results/explain/{explain_job_id}/")
print(f"  - Score: /data/results/score/{score_job_id}/")
print(f"")

# Export comprehensive results
print(f"📦 Exporting comprehensive results...")

try:
    # Download scoring results (JSON and CSV)
    json_response = requests.get(f"{SERVICE_URLS['score']}/api/v1/score/{score_job_id}/download/json")
    if json_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
        json_path = OUTPUT_DIR / f"phi4_layer{TARGET_LAYER}_scoring_results.json"
        with open(json_path, 'wb') as f:
            f.write(json_response.content)
        print(f"✅ Scoring results (JSON): {json_path}")
    
    csv_response = requests.get(f"{SERVICE_URLS['score']}/api/v1/score/{score_job_id}/download/csv")
    if csv_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
        csv_path = OUTPUT_DIR / f"phi4_layer{TARGET_LAYER}_scoring_results.csv"
        with open(csv_path, 'wb') as f:
            f.write(csv_response.content)
        print(f"✅ Scoring results (CSV): {csv_path}")
    
    # Try to export Find results if available
    try:
        find_export_response = requests.get(f"{SERVICE_URLS['find']}/api/v1/find/{find_job_id}/export?format=all")
        if find_export_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
            zip_path = OUTPUT_DIR / f"phi4_layer{TARGET_LAYER}_find_results.zip"
            with open(zip_path, 'wb') as f:
                f.write(find_export_response.content)
            print(f"✅ Find results (ZIP): {zip_path}")
    except:
        print(f"⚠️ Could not export Find results (may not be implemented)")
    
    # Try to export Explain results if available  
    try:
        explain_export_response = requests.get(f"{SERVICE_URLS['explain']}/api/v1/explain/{explain_job_id}/download/all")
        if explain_export_response.status_code in [200, 202]:  # Accept both 200 and 202 as success
            explain_zip_path = OUTPUT_DIR / f"phi4_layer{TARGET_LAYER}_explanations.zip"
            with open(explain_zip_path, 'wb') as f:
                f.write(explain_export_response.content)
            print(f"✅ Explanation results (ZIP): {explain_zip_path}")
    except:
        print(f"⚠️ Could not export Explain results (may not be implemented)")

except Exception as e:
    print(f"⚠️ Error exporting results: {e}")

print(f"")
print(f"✅ WORKFLOW COMPLETE!")
print(f"")
print(f"🔍 To explore your results:")
print(f"  1. Check the {OUTPUT_DIR.name} folder for exported files")
print(f"  2. Review JSON results for detailed feature data")
print(f"  3. Open CSV files in spreadsheet software for analysis")
print(f"  4. Use the job IDs to access results via API endpoints")
print(f"")
print(f"📊 Next steps:")
print(f"  - Analyze feature patterns in the CSV data")
print(f"  - Review explanation quality and coverage")
print(f"  - Use scoring results to identify high-value features")
print(f"  - Consider re-running with different parameters for refinement")
print(f"")
print(f"🎯 Pipeline completed successfully for {MODEL_NAME} layer {TARGET_LAYER}!")


📋 STEP 6: RESULTS SUMMARY AND EXPORT
--------------------------------------------------
🎉 Complete miStudio workflow finished successfully!

📂 Job IDs:
  - Training: train_20250801_035441_8498
  - Feature Analysis: find_20250801_035657_79d4cc16
  - Explanations: explain_20250801_035657_5f5f0d4c
  - Feature Scoring: score_20250801_035727_04625c7c

📁 Results available in shared data directory:
  - Training: /data/results/train/train_20250801_035441_8498/
  - Find: /data/results/find/find_20250801_035657_79d4cc16/
  - Explain: /data/results/explain/explain_20250801_035657_5f5f0d4c/
  - Score: /data/results/score/score_20250801_035727_04625c7c/

📦 Exporting comprehensive results...

✅ WORKFLOW COMPLETE!

🔍 To explore your results:
  1. Check the mistudio_phi4_results folder for exported files
  2. Review JSON results for detailed feature data
  3. Open CSV files in spreadsheet software for analysis
  4. Use the job IDs to access results via API endpoints

📊 Next steps:
  - Analyze feature